In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


## [MYCODE] API Key 셋팅

In [23]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
api_key = os.getenv('OPENAI_API_KEY')
lang_api_key = os.getenv('LANG_API_KEY')

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)



## [LOG] 웹페이지 태그 확인하고 출력해보기

In [12]:
# 자동으로 제목과 본문을 추출하는 코드 예시
from bs4 import BeautifulSoup
import requests

def extract_title_and_content(url):
    # 1. URL에서 HTML 가져오기
    response = requests.get(url)
    html = response.text

    # 2. BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(html, "html.parser")

  # 제목 추출 (h1, h2, h3 태그 중 첫 번째) + get_text() 사용
    title_tag = soup.find(["h1", "h2", "h3"])
    title = title_tag.get_text(strip=True) if title_tag else "No Title"


    # 4. editedContent 클래스의 <div> 태그 찾기
    content_div = soup.find("div", class_="editedContent")
    content = content_div.get_text(strip=True) if content_div else "No Content Found"

    # 5. 결과 반환
    return {
        "title": title,
        "content": content
    }

# 테스트 실행
url = "https://spartacodingclub.kr/blog/all-in-challenge_winner"
result = extract_title_and_content(url)

# 출력
print("Title:", result["title"])
print("Content:", result["content"])



Title: 'AII-in 코딩 공모전’ 수상작을 소개합니다
Content: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어문학을 공부하며 학습 속도도 눈에 띄게 빨라질 수 있어요. 언어 공부의 복잡함을 단순하게 만들어주는 Lexi No

## [MYCODE] html 분석을 바탕으로 RAG 데이터 만들고 프롬프트 작성하기

In [71]:
from langchain.schema import Document

title_text = result["title"]
content_text = result["content"]

# LangChain Document 객체 생성
doc = Document(
    page_content=content_text,
    metadata={"title": title_text, "source": url}
)

# 출력 확인
print("Title:", doc.metadata["title"])
print("Content:", doc.page_content[:500])  # 앞 500자 출력

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

docs_chunks = splitter.split_documents([doc])
print("_"*30 + "docs_chunks" + "-"*30)
# 출력 확인
for chunk in docs_chunks[:3]:  # 첫 3개 청크 출력
    print("Chunk Content:", chunk.page_content)

vectorstore = Chroma.from_documents(
    documents=docs_chunks,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

retriever = vectorstore.as_retriever()

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs_chunks)
prompt = hub.pull("rlm/rag-prompt")

user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print("_"*30 + "prompt" + "-"*30)
print(user_prompt)


Title: 'AII-in 코딩 공모전’ 수상작을 소개합니다
Content: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀
______________________________docs_chunks------------------------------
Chunk Content: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆 대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전

/Users/yunhyeokchoi/.pyenv/versions/3.11.10/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


______________________________prompt------------------------------
messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: ALL-in 코딩 공모전 수상작들을 요약해줘. \nContext: 코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.🏆\xa0대상[Lexi Note] 언어공부 필기 웹 서비스서비스 제작자: 다나와(김다애, 박나경)💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로\n\n뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.W와 같이 

## [LOG] 최종 결과 확인하기

In [72]:
response = llm.invoke(user_prompt)
print(response.content)

ALL-in 코딩 공모전에서는 대학생들이 캠퍼스에서의 문제를 창의적으로 해결하기 위해 다양한 웹/앱 서비스를 개발했습니다. 수상작 중에는 언어학습을 돕는 'Lexi Note', 벌레 퇴치 매칭 서비스 '우리집 히어로즈', 실시간 소통이 가능한 '에코 클래스룸', 동아리 정보 플랫폼 'Crewing', 학교생활 관리 앱 '학교생활 매니저', 그리고 외국인과 내국인 매칭 플랫폼 'BLOTIE'가 있습니다. 이들 서비스는 각기 다른 문제를 해결하고, 효율적인 대학 생활을 지원합니다.
